In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vqa-dataset/vqa_dataset/data-00016-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00004-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/state.json
/kaggle/input/vqa-dataset/vqa_dataset/data-00011-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/dataset_info.json
/kaggle/input/vqa-dataset/vqa_dataset/data-00002-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00028-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00015-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00013-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00019-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00027-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00031-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00006-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00033-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00008-of-00034.arrow
/kaggle/input/vqa-dataset/vqa_dataset/data-00025-of-00034.arrow

In [2]:
from datasets import load_from_disk
dataset = load_from_disk("/kaggle/input/vqa-dataset/vqa_dataset")

Loading dataset from disk:   0%|          | 0/34 [00:00<?, ?it/s]

In [3]:
dataset

Dataset({
    features: ['question_type', 'multiple_choice_answer', 'answers', 'image_id', 'answer_type', 'question_id', 'question', 'image'],
    num_rows: 110939
})

In [4]:
dataset[0]

{'question_type': 'what is this',
 'multiple_choice_answer': 'net',
 'answers': [{'answer': 'net', 'answer_confidence': 'maybe', 'answer_id': 1},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 2},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 3},
  {'answer': 'netting', 'answer_confidence': 'yes', 'answer_id': 4},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 5},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 6},
  {'answer': 'mesh', 'answer_confidence': 'maybe', 'answer_id': 7},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 8},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 9},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 10}],
 'image_id': 458752,
 'answer_type': 'other',
 'question_id': 458752000,
 'question': 'What is this photo taken looking through?',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>}

In [ ]:
dataset[2000]['image']

In [6]:
!pip install pytorch-pretrained-vit
!pip install vit-pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for pytorch-pretrained-vit: filename=pytorch_pretrained_vit-0.0.7-py3-none-any.whl size=11116 sha256=b1c71f48246314f7db431d176107a99cb8bdacd0b1a366b09d725046ae165099
  Stored in directory: /root/.cache/pip/wheels/2d/46/ad/12007be9d377d0fbf27ef75b6e47ed92832ab6b70dbf004b6f
Successfully built pytorch-pretrained-vit
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.3 MB/s eta 0:00:00


In [7]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from vit_pytorch import ViT
from torch.utils.data import DataLoader, Dataset
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
class VQADataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image_path = item["image"]
        question = item["question"]
        label = item["multiple_choice_answer"]

        # Preprocess image
        image = preprocess_image(image_path)
        # Tokenize inputs
        inputs = self.tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=64)
        input_ids = inputs["input_ids"].squeeze(0)

        # Create attention mask with the same shape as input_ids
        attention_mask = (input_ids != 0).long()
        return {"input_ids": input_ids, "attention_mask": attention_mask, "image": image, "label": label}


In [9]:
print(tokenizer.vocab_size)

30522


In [10]:
def preprocess_image(image_path):
    image = image_path
    if image.mode != 'RGB':
        image = image.convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = transform(image)
    return image

def show_image(image):
    # Denormalize the image
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    # Convert tensor to numpy array
    image = image.cpu().numpy().transpose(1, 2, 0)
    
    # Denormalize
    image = std * image + mean
    image = np.clip(image, 0, 1)

    # Show the image
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [16]:
for i, batch in enumerate(dataloader):
    if i %1000:
        continue
    else:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        image = batch["image"].to(device)
        question = tokenizer.decode(input_ids.squeeze().tolist())  # Convert input_ids to string question

        # Debug: Print shape of the image tensor
        print("Image shape:", image.shape)

        show_image(image.squeeze())

        print("Question:", question)
        print("Answer:", batch['label'])

NameError: name 'dataloader' is not defined

 # LoRA finetunig

In [11]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.6 MB/s eta 0:00:00a 0:00:01


In [31]:
import torch
from transformers import BertModel, ViTModel

# Load the models
bert_model = BertModel.from_pretrained('bert-base-uncased')
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# Print module names in BERT model
print("BERT Model Modules:")
for name, module in bert_model.named_modules():
    print(name)

# Print module names in ViT model
print("\nViT Model Modules:")
for name, module in vit_model.named_modules():
    print(name)


BERT Model Modules:

embeddings
embeddings.word_embeddings
embeddings.position_embeddings
embeddings.token_type_embeddings
embeddings.LayerNorm
embeddings.dropout
encoder
encoder.layer
encoder.layer.0
encoder.layer.0.attention
encoder.layer.0.attention.self
encoder.layer.0.attention.self.query
encoder.layer.0.attention.self.key
encoder.layer.0.attention.self.value
encoder.layer.0.attention.self.dropout
encoder.layer.0.attention.output
encoder.layer.0.attention.output.dense
encoder.layer.0.attention.output.LayerNorm
encoder.layer.0.attention.output.dropout
encoder.layer.0.intermediate
encoder.layer.0.intermediate.dense
encoder.layer.0.intermediate.intermediate_act_fn
encoder.layer.0.output
encoder.layer.0.output.dense
encoder.layer.0.output.LayerNorm
encoder.layer.0.output.dropout
encoder.layer.1
encoder.layer.1.attention
encoder.layer.1.attention.self
encoder.layer.1.attention.self.query
encoder.layer.1.attention.self.key
encoder.layer.1.attention.self.value
encoder.layer.1.attention.s

In [12]:
%%time
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from transformers import BertModel, ViTModel, BertTokenizer

# Assuming dataset and VQADataset are already defined
small_dataset = dataset.select(range(10000))

# Split the dataset into training, validation, and test sets
train_size = int(0.8 * len(small_dataset))
val_size = int(0.1 * len(small_dataset))
test_size = len(small_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(small_dataset, [train_size, val_size, test_size])

# Create VQADataset instances
train_vqa_dataset = VQADataset(train_dataset, tokenizer)
val_vqa_dataset = VQADataset(val_dataset, tokenizer)
test_vqa_dataset = VQADataset(test_dataset, tokenizer)

# Custom collate function
def custom_collate_fn(batch):
    max_seq_length = max(len(item["input_ids"]) for item in batch)
    
    padded_input_ids = []
    padded_attention_masks = []
    images = []
    labels = []
    
    for item in batch:
        input_ids = item["input_ids"]
        attention_mask = item["attention_mask"]
        image = item["image"]
        label = item["label"]
        
        # Pad input_ids and attention_mask
        padding_length = max_seq_length - len(input_ids)
        padded_input_ids.append(torch.cat([input_ids, torch.zeros(padding_length, dtype=torch.long)]))
        padded_attention_masks.append(torch.cat([attention_mask, torch.zeros(padding_length, dtype=torch.long)]))
        
        images.append(image)
        
        # Encode labels
        encoded_label = torch.tensor(tokenizer.convert_tokens_to_ids(label))
        labels.append(encoded_label)
    
    padded_input_ids = torch.stack(padded_input_ids)
    padded_attention_masks = torch.stack(padded_attention_masks)
    images = torch.stack(images)
    labels = torch.stack(labels)  # No need for this to be a list
    
    return {
        "input_ids": padded_input_ids,
        "attention_mask": padded_attention_masks,
        "image": images,
        "label": labels
    }

# Create DataLoader instances with custom collate function
train_loader = DataLoader(train_vqa_dataset, batch_size=64, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_vqa_dataset, batch_size=64, shuffle=False, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_vqa_dataset, batch_size=64, shuffle=False, collate_fn=custom_collate_fn)


CPU times: user 12.1 ms, sys: 3.02 ms, total: 15.1 ms
Wall time: 32.2 ms


In [15]:
%%time
import torch
import torch.nn as nn
from transformers import BertModel, ViTModel, BertTokenizer
from peft import LoraConfig, get_peft_model
from torch.utils.data import DataLoader
from tqdm import tqdm

class VQAModel(nn.Module):
    def __init__(self, image_model, text_model, vocab_size):
        super(VQAModel, self).__init__()
        self.image_model = image_model
        self.text_model = text_model
        
        # LoRA configuration
        self.lora_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.1, bias="none")
        
        # Apply LoRA to BERT and ViT models
        self.text_model = get_peft_model(
            self.text_model,
            self.lora_config
        )
        self.image_model = get_peft_model(
            self.image_model,
            self.lora_config
        )

        # Transformer decoder layer
        self.decoder_layer = nn.TransformerDecoderLayer(d_model=768, nhead=8)
        self.transformer_decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=6)

        # Final classification layer
        self.fc = nn.Linear(768, vocab_size)

    def forward(self, image, input_ids, attention_mask):
        # Extract image features using ViT
        image_output = self.image_model(pixel_values=image).last_hidden_state  # Shape: [batch_size, seq_len, 768]
        image_cls_token = image_output[:, 0, :]  # Use [CLS] token representation for the image

        # Expand image features to match the batch size
        weighted_image_output = image_cls_token.unsqueeze(1)  # [batch_size, 1, 768]

        # Extract text features
        text_output = self.text_model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state

        # Prepare for transformer decoder
        image_features = weighted_image_output.permute(1, 0, 2)  # [1, batch_size, 768]
        text_features = text_output.permute(1, 0, 2)  # [seq_len, batch_size, 768]

        # Transformer decoder forward pass
        decoder_output = self.transformer_decoder(tgt=text_features, memory=image_features)

        # Classification
        combined_output = decoder_output.permute(1, 0, 2)  # [batch_size, seq_len, 768]
        logits = self.fc(combined_output[:, -1, :])  # Use the last token's representation for classification

        return logits

# Set up the model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
text_model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
vocab_size = tokenizer.vocab_size

model = VQAModel(image_model, text_model, vocab_size).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

# Training loop
epochs =15
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        image = batch["image"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(image, input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

# Evaluation loop
    model.eval()
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            image = batch["image"].to(device)
            labels = batch["label"]

            outputs = model(image, input_ids, attention_mask)
            predictions = torch.argmax(outputs, dim=-1).tolist()
            all_predictions.extend(predictions)
            all_labels.extend(labels.tolist())

    # Compute accuracy
    correct = sum(1 for pred, label in zip(all_predictions, all_labels) if pred == label)
    accuracy = correct / len(all_labels)
    print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1/15: 100%|██████████| 125/125 [03:26<00:00,  1.65s/it]


Epoch 1/15, Loss: 4.4449


Validation: 100%|██████████| 16/16 [00:14<00:00,  1.07it/s]


Validation Accuracy: 0.3480


Epoch 2/15: 100%|██████████| 125/125 [02:55<00:00,  1.41s/it]


...

Validation Accuracy: 0.4050


Epoch 15/15: 100%|██████████| 125/125 [02:54<00:00,  1.40s/it]


Epoch 15/15, Loss: 0.2057


Validation: 100%|██████████| 16/16 [00:13<00:00,  1.16it/s]

Validation Accuracy: 0.3820
CPU times: user 58min 16s, sys: 5min 25s, total: 1h 3min 42s
Wall time: 47min 40s


In [18]:
def decode_predictions(outputs, tokenizer):
    probabilities = torch.softmax(outputs, dim=-1)
    predicted_indices = torch.argmax(probabilities, dim=-1)
    predictions = []
    for indices in predicted_indices:
        tokens = tokenizer.convert_ids_to_tokens(indices.tolist())
        prediction = "".join(tokens)
        predictions.append(prediction)
    return predictions


In [19]:
from collections import defaultdict

def compute_accuracy(predictions, labels):
    correct = sum(1 for pred, label in zip(predictions, labels) if pred == label)
    total = len(labels)
    return correct / total


def compute_precision_recall_f1(predictions, labels):
    class_tp = defaultdict(int)
    class_fp = defaultdict(int)
    class_fn = defaultdict(int)

    for pred, label in zip(predictions, labels):
        if pred == label:
            class_tp[label] += 1
        else:
            class_fp[pred] += 1
            class_fn[label] += 1

    all_classes = set(class_tp.keys()).union(class_fp.keys()).union(class_fn.keys())

    precisions = []
    recalls = []
    f1_scores = []

    for cls in all_classes:
        tp = class_tp[cls]
        fp = class_fp[cls]
        fn = class_fn[cls]

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    micro_precision = sum(class_tp.values()) / (sum(class_tp.values()) + sum(class_fp.values())) if (sum(class_tp.values()) + sum(class_fp.values())) > 0 else 0
    micro_recall = sum(class_tp.values()) / (sum(class_tp.values()) + sum(class_fn.values())) if (sum(class_tp.values()) + sum(class_fn.values())) > 0 else 0
    micro_f1 = 2 * micro_precision * micro_recall / (micro_precision + micro_recall) if (micro_precision + micro_recall) > 0 else 0

    macro_precision = sum(precisions) / len(precisions) if precisions else 0
    macro_recall = sum(recalls) / len(recalls) if recalls else 0
    macro_f1 = sum(f1_scores) / len(f1_scores) if f1_scores else 0

    return micro_precision, micro_recall, micro_f1, macro_precision, macro_recall, macro_f1


In [13]:
print('hi')

hi


In [25]:
# Set the model to evaluation mode
model.eval()

# Define lists to store predictions and ground truth labels
all_predictions = []
all_labels = []

# Iterate through the test dataset
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        image = batch["image"].to(device)
        labels = batch["label"]
#         print(labels)
        
        # Forward pass
        outputs = model(image, input_ids, attention_mask)
#         print(torch.argmax(outputs, dim=-1))
        # Decode predictions
        predictions = decode_predictions(outputs,tokenizer) 
        lbl = []
        for indices in labels:
            tokens = tokenizer.convert_ids_to_tokens(indices.tolist())
            prediction = "".join(tokens)
            lbl.append(prediction)
        # Collect predictions and ground truth labels
        all_predictions.extend(predictions)
        all_labels.extend(lbl)

# Compute evaluation metrics
print(all_predictions)
print(all_labels)
accuracy = compute_accuracy(np.array(all_predictions), np.array(all_labels))
micro_precision, micro_recall, micro_f1,macro_precision, macro_recall, macro_f1 = compute_precision_recall_f1(np.array(all_predictions), np.array(all_labels))

print(f"Accuracy: {accuracy:.4f}")
print(f"Micro Precision: {micro_precision:.4f}")
print(f"Micro Recall: {micro_recall:.4f}")
print(f"Micro F1 Score: {micro_f1:.4f}")
print(f"Macro Precision: {micro_precision:.4f}")
print(f"Macro Recall: {micro_recall:.4f}")
print(f"Macro F1 Score: {micro_f1:.4f}")

['[UNK]', 'yes', 'thai', 'no', 'no', 'gray', '[UNK]', '[UNK]', 'yes', 'resting', 'tree', 'no', '6', 'yes', 'fork', '[UNK]', '1', 'cow', 'yes', '[UNK]', '[UNK]', 'water', 'cow', 'yes', '[UNK]', '[UNK]', 'no', '[UNK]', 'kitchen', '[UNK]', '2', 'no', 'yes', 'yes', 'blue', 'blue', '[UNK]', 'no', 'yes', '[UNK]', 'no', 'yes', 'yes', '7', 'blue', 'red', '0', 'yes', 'white', '0', '0', '6', 'bear', 'yellow', 'eating', 'boat', '[UNK]', '1', '2', 'no', '[UNK]', 'yes', '2', 'white', 'sleeping', 'yes', '[UNK]', '[UNK]', '[UNK]', 'phone', 'yes', '2', 'no', '[UNK]', '[UNK]', '7', 'outside', 'brown', '6', 'calf', '[UNK]', '[UNK]', '[UNK]', 'no', 'gray', '[UNK]', 'field', '[UNK]', 'yes', 'right', 'no', 'yes', 'blue', '[UNK]', '[UNK]', '[UNK]', 'no', 'yes', '[UNK]', '3', 'chair', '4', 'yes', 'no', 'snow', '4', 'yes', 'black', 'chocolate', 'brown', 'wood', 'cat', 'no', 'yes', 'yes', 'no', 'surfing', 'red', 'motorola', 'yes', 'red', '[UNK]', 'black', 'yes', 'leaves', 'tile', 'yes', '[UNK]', 'pizza', '2', 

In [16]:
import os

# Define the path to save the model
save_path = "/kaggle/working/vqa_model.pth"

# Save the model
torch.save(model.state_dict(), save_path)

# Check if the file is saved
if os.path.exists(save_path):
    print("Model saved successfully at:", save_path)
else:
    print("Failed to save the model.")


Model saved successfully at: /kaggle/working/vqa_model.pth


In [17]:
import pickle

# Define the path to save the model
save_path = "/kaggle/working/vqa_model.pkl"

# Save the model
with open(save_path, 'wb') as f:
    pickle.dump(model.state_dict(), f)

    
# Check if the file is saved
if os.path.exists(save_path):
    print("Model saved successfully at:", save_path)
else:
    print("Failed to save the model.")


Model saved successfully at: /kaggle/working/vqa_model.pkl
